# Часові ряди

In [378]:
import numpy as np

In [379]:
u = 0.1
y = np.array( [ 4*t+3+np.sin(t) + u for t in range(1, 50)] )
test = y[47:49]
y = y[:47]

In [380]:
y, test

(array([  7.94147098,  12.00929743,  15.24112001,  18.3431975 ,
         22.14107573,  26.8205845 ,  31.7569866 ,  36.08935825,
         39.51211849,  42.55597889,  46.10000979,  50.56342708,
         55.52016704,  60.09060736,  63.75028784,  66.81209668,
         70.13860251,  74.34901275,  79.24987721,  84.01294525,
         87.93665564,  91.09114869,  94.2537796 ,  98.19442164,
        102.96764825, 107.86255845, 112.05637593, 115.37090579,
        118.43636612, 122.11196838, 126.69596235, 131.65142668,
        136.09991186, 139.62908269, 142.67181733, 146.10822115,
        150.45646187, 155.39636858, 160.06379539, 163.84511316,
        166.94137733, 170.18347845, 174.26822526, 179.11770193,
        183.95090352, 188.00178835, 191.22357312]),
 array([194.33174534, 198.14624735]))

### 0) Визначення тренду

#### Тест серій, заснований на медіані

$$y_{med}= \left\{\begin{matrix}
 y_{\frac{n+1}{2}} & якщо n - непарне\\
 \frac{1}{2}(y_{\frac{n}{2}}+y_{\frac{n}{2}+1}) & якщо n - парне
\end{matrix}\right. $$

In [381]:
def get_med(y):
    y_med = 0
    if len(y) % 2:
        print("Непарне значення n")
        y_med = y[(len(y)+1)//2]-1
    else:
        print("Парне значення n")
        y_med = ( y[len(y)//2-1]+y[len(y)//2] ) / 2
    return y_med

Алгорітм тесту перевірка 
1. Якщо $y_t > y_{med}$, ставимо плюс;
2. Якщо $y_t < y_{med}$, ставимо плюс;
3. Якщо $y_t = y_{med}$, не враховуємо

Для існування тенденції чергування плюсiв i мiнусiв у послiдовностi повинне бути бiльш-менш випадковим, тобто ця послiдовнiсть не повинна мати досить довгих серiй плюсiв або мiнусiв i, вiдповiдно, загальна кiлькiсть серiй не повинна бути дуже малою.

In [382]:
def test_series(y):
    y_med = get_med(y)
    test_arr = [ -1*(t < y_med) + (t>y_med) for t in y]
    if test_arr.count(0):
        test_arr = test_arr[test_arr != 0]
    series = 0
    count_series = 1
    longest_series = 0
    for i in range(1, len(test_arr)):
        if test_arr[i-1] == test_arr[i]:
            series += 1
        else:
            if longest_series < series:
                longest_series = series
            series = 0
            count_series += 1
    check_count = int(1/2*( len(y) + 2 -1.96*np.sqrt(len(y)-1))) 
    check_series = int( 1.43 * np.log(len(y) + 1 ) )
    if count_series < check_count or longest_series > check_series:
        return 1
    return 0

In [383]:
test_series(y)

Непарне значення n


1

Рівності виконуються, тому тенденція існує 95% відсотків

#### Тест Фостера - Стюарта

Крiм тенденцiї у середньому вiн дозволяє встановити наявнiсть тенденцiї дисперсiї ряду

In [384]:
def get_k(y):
    """
    k_t = 1, якщо y_t більше за всі попередні значення послідовності, інакше 0
    """
    k_t = []
    max_y = y[0]
    for i in range(1, len(y)):
        if max_y < y[i]:
            k_t.append(1)
            max_y = y[i]
        else:
            k_t.append(0)
    return np.array( k_t )

In [385]:
k = get_k([5, 8, 6, 7, 7, 10, 4])
k

array([1, 0, 0, 0, 1, 0])

In [386]:
def get_l(y):
    """
    l_t = 1, якщо y_t менше за всі попередні значення послідовності, інакше 0
    """
    l_t = []
    min_y = y[0]
    for i in range(1, len(y)):
        if min_y > y[i]:
            l_t.append(1)
            min_y = y[i]
        else:
            l_t.append(0)
    return np.array( l_t )

In [387]:
l = get_l([5, 8, 6, 7, 7, 10, 4])
l

array([0, 0, 0, 0, 0, 1])

1. Величина s характеризує зміну часового ряду дисперсії рівнів ряду і змінюється від 0 до (n-1)
2. Величина d характеризує зміну у середньому і змінюється від -(n-1) до (n-1) 
Если пожалуется, то поправится насчет характеристик

In [388]:
def calc_s_and_d(k, l):
    """
    Розрахування величин s та d
    """
    k = k[1:]
    l = l[1:]
    return sum(k+l), sum(k-l)

In [389]:
calc_s_and_d(k, l)

(2, 0)

In [390]:
def test_foster_stuart(y):
    k = get_k(y)
    l = get_l(y)
    s, d = calc_s_and_d(k, l)
    mu = 6.790 # математичне очікування величини s для ряду, в якому рівні розташовані випадковим чином
    sigma_1 = np.sqrt(2*np.log(len(y)-3.4253))
    sigma_2 = np.sqrt(2*np.log(len(y)-0.8456))
    t_s = np.abs(s-mu)/sigma_1
    t_d = np.abs(d-0)/sigma_2 # відхилення величини d від нуля
    # тест Ст'юдента
    t_stud = 1.6787 # a 0.1, n=47
    return t_d > t_stud, t_s > t_stud 

In [391]:
test_foster_stuart(y)

(True, True)

З вірогітностью 90% існує тенденція у середньому (є чинники довгочасові) та дисперсії (є чинники сезонні)

### 1) Згладжуємо часовий ряд ковзною середньою

Ковзне середнє

$$\bar{y}_t=\frac{y_{t-1}+y_t+y_{t+1}}{3}$$
Щоб не загубити перший і останній рівні, то їх згладжують за формулою
$$\bar{y}_1=\frac{5y_{1}+2y_2-y_{3}}{3}$$
$$\bar{y}_n=\frac{-y_{n-2}+2y_{n-1}+5y_{n}}{3}$$

In [392]:
def mv_avg(y):
    y_avg_1 = (5*y[0]+2*y[1]-y[2])/6 # щоб не загубити рівні
    y_avg = [ (y[i-1]+y[i]+y[i+1])/3 for i in range(1, len(y) - 1)]
    y_avg_n = (5*y[-1]+2*y[-2]-y[-3])/6
    return [y_avg_1] + y_avg + [y_avg_n]

Перші прирости

$$\bar{u}^1_t=\frac{\bar{y}_{t+1}-\bar{y}_{t-1}}{2}$$

In [393]:
def calc_avg_incr(y):
    return [ (y[i]- y[i-1])/2 for i in range(1, len(y))]

In [394]:
y_avg = mv_avg(y)
u1 = calc_avg_incr(y_avg)
u1_y1 = [u1[i]/y_avg[i] for i in range(1, len(y)-1)]
ln_u1 = np.log(u1[1:])
ln_u1_y1 = np.log(u1_y1)

In [395]:
print("y_avrg:", np.round(y_avg, 3))
print("---------------")
print("u1:" , np.round(u1, 3))
print("---------------")

y_avrg: [  8.081  11.731  15.198  18.575  22.435  26.906  31.556  35.786  39.386
  42.723  46.406  50.728  55.391  59.787  63.551  66.9    70.433  74.579
  79.204  83.733  87.68   91.094  94.513  98.472 103.008 107.629 111.763
 115.288 118.64  122.415 126.82  131.482 135.793 139.467 142.803 146.412
 150.654 155.306 159.768 163.617 166.99  170.464 174.523 179.112 183.69
 187.725 191.362]
---------------
u1: [1.825 1.734 1.689 1.93  2.236 2.325 2.115 1.8   1.668 1.842 2.161 2.332
 2.198 1.882 1.675 1.766 2.073 2.312 2.265 1.974 1.707 1.71  1.979 2.268
 2.31  2.067 1.762 1.676 1.888 2.203 2.331 2.156 1.837 1.668 1.805 2.121
 2.326 2.231 1.924 1.687 1.737 2.029 2.295 2.289 2.018 1.818]
---------------


In [396]:
print("u1/y_avrg:", np.round(u1_y1, 3))
print("---------------")
print("ln(u1):", np.round(ln_u1, 3))
print("---------------")
print("ln(u1/y_avrg):", np.round(ln_u1_y1, 3))
print("---------------")

u1/y_avrg: [0.148 0.111 0.104 0.1   0.086 0.067 0.05  0.042 0.043 0.047 0.046 0.04
 0.031 0.026 0.026 0.029 0.031 0.029 0.024 0.019 0.019 0.021 0.023 0.022
 0.019 0.016 0.015 0.016 0.018 0.018 0.016 0.014 0.012 0.013 0.014 0.015
 0.014 0.012 0.01  0.01  0.012 0.013 0.013 0.011 0.01 ]
---------------
ln(u1): [0.55  0.524 0.657 0.805 0.844 0.749 0.588 0.512 0.611 0.77  0.847 0.787
 0.632 0.516 0.569 0.729 0.838 0.817 0.68  0.535 0.536 0.683 0.819 0.837
 0.726 0.567 0.516 0.635 0.79  0.846 0.768 0.608 0.512 0.59  0.752 0.844
 0.803 0.654 0.523 0.552 0.708 0.831 0.828 0.702 0.598]
---------------
ln(u1/y_avrg): [-1.912 -2.197 -2.264 -2.306 -2.449 -2.703 -2.99  -3.162 -3.144 -3.067
 -3.08  -3.227 -3.458 -3.636 -3.634 -3.526 -3.474 -3.555 -3.748 -3.939
 -3.976 -3.866 -3.771 -3.797 -3.952 -4.15  -4.231 -4.141 -4.018 -3.996
 -4.111 -4.303 -4.426 -4.371 -4.235 -4.171 -4.243 -4.419 -4.575 -4.566
 -4.431 -4.332 -4.36  -4.511 -4.637]
---------------


Так як перші прирости та ділення $\frac{u_t}{y_t}$ майже однакові між собою, то можемо обрати або поліном першого порядку, або просту експоненту. Також можна побачити за приростами сезоність, далі її будемо оцінювати

Оберемо для тендценції поліном першого порядку $a_0+a_1t$

### 2) Оцінюємо сезонні коливання (S)

In [397]:
abs_avg = y-y_avg

print("Абсолютні значення:", np.round(abs_avg, 3))
print("Відносні значення:", np.round((y-y_avg)/y, 3))

Абсолютні значення: [-0.139  0.279  0.043 -0.232 -0.294 -0.086  0.201  0.303  0.126 -0.167
 -0.306 -0.164  0.129  0.304  0.199 -0.088 -0.295 -0.23   0.046  0.28
  0.256 -0.003 -0.259 -0.278 -0.041  0.234  0.293  0.083 -0.203 -0.303
 -0.124  0.169  0.306  0.162 -0.131 -0.304 -0.197  0.091  0.295  0.228
 -0.049 -0.281 -0.255  0.005  0.261  0.276 -0.138]
Відносні значення: [-0.018  0.023  0.003 -0.013 -0.013 -0.003  0.006  0.008  0.003 -0.004
 -0.007 -0.003  0.002  0.005  0.003 -0.001 -0.004 -0.003  0.001  0.003
  0.003 -0.    -0.003 -0.003 -0.     0.002  0.003  0.001 -0.002 -0.002
 -0.001  0.001  0.002  0.001 -0.001 -0.002 -0.001  0.001  0.002  0.001
 -0.    -0.002 -0.001  0.     0.001  0.001 -0.001]


Так як відносні абсолютні відхилення змінюються приблизно на одному рівні між собою, то свідчить про адитивний тип зв'язку тренду і сезоної компоненти

$$S$$

In [398]:
quad_3 = np.array([ np.sum(abs_avg[2*(i-1):2*i]) for i in range(1, round(len(y_avg)/3))]) # всього за i квартал (ковзне середнє при 3) 
quad_3

array([ 0.13933398, -0.18868521, -0.37950796,  0.50454729, -0.04042356,
       -0.47090302,  0.43235316,  0.11105822, -0.52478621,  0.32571802,
        0.25369316, -0.53686524,  0.19313638,  0.37611905, -0.50617788])

In [399]:
avg_quad_3 = quad_3/3

In [400]:
np.sum(avg_quad_3)

-0.10379660746041054

In [401]:
k = np.sum(avg_quad_3)/len(quad_3)
k*15

-0.10379660746041054

Перевіримо умову рівності нулеві суми значень скориговних сезонних компонент, що вірним (python шаліт)

In [402]:
np.sum(quad_3) - k*15

-0.20759321492082117

Отримали такі значення сезонної компоненти

$$\hat{S_t}$$

In [403]:
S = quad_3-k
S = np.concatenate((np.tile(S, 3), quad_3[:2]))
S

array([ 0.14625375, -0.18176544, -0.37258818,  0.51146706, -0.03350378,
       -0.46398324,  0.43927293,  0.11797799, -0.51786644,  0.3326378 ,
        0.26061293, -0.52994546,  0.20005615,  0.38303882, -0.49925811,
        0.14625375, -0.18176544, -0.37258818,  0.51146706, -0.03350378,
       -0.46398324,  0.43927293,  0.11797799, -0.51786644,  0.3326378 ,
        0.26061293, -0.52994546,  0.20005615,  0.38303882, -0.49925811,
        0.14625375, -0.18176544, -0.37258818,  0.51146706, -0.03350378,
       -0.46398324,  0.43927293,  0.11797799, -0.51786644,  0.3326378 ,
        0.26061293, -0.52994546,  0.20005615,  0.38303882, -0.49925811,
        0.13933398, -0.18868521])

### 3) Знаходимо значення тенденції із залишками

$$T_t+\varepsilon_t=y_t-\hat{S_t}$$

In [404]:
Tt_et = y - S
Tt_et

array([  7.79521723,  12.19106287,  15.61370819,  17.83173044,
        22.17457951,  27.28456774,  31.31771367,  35.97138025,
        40.02998492,  42.22334109,  45.83939686,  51.09337254,
        55.32011089,  59.70756853,  64.24954595,  66.66584293,
        70.32036795,  74.72160094,  78.73841015,  84.04644903,
        88.40063888,  90.65187576,  94.1358016 ,  98.71228807,
       102.63501045, 107.60194552, 112.58632139, 115.17084964,
       118.05332729, 122.61122649, 126.5497086 , 131.83319212,
       136.47250005, 139.11761563, 142.70532111, 146.57220439,
       150.01718894, 155.27839058, 160.58166182, 163.51247536,
       166.6807644 , 170.71342391, 174.06816911, 178.7346631 ,
       184.45016163, 187.86245437, 191.41225834])

### 4) Визначаємо аналітичний вигляд тенденції (T)

Використаємо метод найменших квадратів для $a_0+a_1t; t=\bar{ 1, 47 }$

In [405]:
t = np.array(range(1, 48))

# Расчет коэффициентов
A = np.vstack([t, np.ones(len(t))]).T
a_1, a_0 = np.linalg.lstsq(A, Tt_et, rcond=None)[0]
a_1, a_0

(4.000447703825212, 3.143667666692438)

$$a_0=4, a_1=3.144$$

### 5) Розрахуовуємо аналітичний вигляд тенденції

$$T = 4 + 3.144t$$

In [406]:
T = a_0 + a_1 * t
T

array([  7.14411537,  11.14456307,  15.14501078,  19.14545848,
        23.14590619,  27.14635389,  31.14680159,  35.1472493 ,
        39.147697  ,  43.1481447 ,  47.14859241,  51.14904011,
        55.14948782,  59.14993552,  63.15038322,  67.15083093,
        71.15127863,  75.15172634,  79.15217404,  83.15262174,
        87.15306945,  91.15351715,  95.15396485,  99.15441256,
       103.15486026, 107.15530797, 111.15575567, 115.15620337,
       119.15665108, 123.15709878, 127.15754649, 131.15799419,
       135.15844189, 139.1588896 , 143.1593373 , 147.159785  ,
       151.16023271, 155.16068041, 159.16112812, 163.16157582,
       167.16202352, 171.16247123, 175.16291893, 179.16336664,
       183.16381434, 187.16426204, 191.16470975])

### 6) Знаходимо суму (T+S)

In [407]:
T_S = T + S
T_S

array([  7.29036912,  10.96279764,  14.77242259,  19.65692554,
        23.1124024 ,  26.68237065,  31.58607452,  35.26522729,
        38.62983056,  43.4807825 ,  47.40920534,  50.61909465,
        55.34954397,  59.53297434,  62.65112511,  67.29708468,
        70.96951319,  74.77913815,  79.6636411 ,  83.11911796,
        86.6890862 ,  91.59279008,  95.27194285,  98.63654612,
       103.48749806, 107.4159209 , 110.62581021, 115.35625953,
       119.5396899 , 122.65784067, 127.30380024, 130.97622875,
       134.78585371, 139.67035666, 143.12583352, 146.69580176,
       151.59950564, 155.27865841, 158.64326168, 163.49421362,
       167.42263646, 170.63252577, 175.36297508, 179.54640546,
       182.66455623, 187.30359602, 190.97602453])

### 7) Розраховуємо залишки

$$\varepsilon_t=y_t-\hat{S_t}-T_t$$

In [408]:
e = y - T_S
e

array([ 0.65110186,  1.04649979,  0.46869741, -1.31372804, -0.97132668,
        0.13821386,  0.17091207,  0.82413096,  0.88228792, -0.92480361,
       -1.30919555, -0.05566757,  0.17062307,  0.55763301,  1.09916273,
       -0.484988  , -0.83091068, -0.4301254 , -0.41376389,  0.89382729,
        1.24756943, -0.50164139, -1.01816325, -0.44212448, -0.51984981,
        0.44663755,  1.43056572,  0.01464626, -1.10332379, -0.5458723 ,
       -0.60783788,  0.67519793,  1.31405815, -0.04127397, -0.45401619,
       -0.58758061, -1.14304377,  0.11771017,  1.42053371,  0.35089954,
       -0.48125913, -0.44904731, -1.09474983, -0.42870353,  1.2863473 ,
        0.69819233,  0.24754859])

### 8) Перевіряємо модель на адекватність

### 9) Робимо прогноз за отриманою моделлю

$$RSS = \sum_{47}^{1}\varepsilon^2_t$$  

In [417]:
e2 = np.average(e)

In [418]:
check1 = a_0 + a_1 * 48 + S[2] 
check2 = a_0 + a_1 * 48 + S[3]
print("test1:", check1, test[0], "\ntest2:", check2, test[1])

test1: 194.7925692653756 194.3317453386763 
test2: 195.676624510233 198.14624734724052


In [411]:
RSS = np.sum(np.square(e))
RSS

29.772978129768315

### 10) Оцінюємо точність прогнозу

### 11) Визначення тренду